In [ ]:
import os
import sys
sys.path.append(os.path.abspath(".."))
from collections import defaultdict, Counter

from speclib.loaders import loadUser, loadUserPhonenumberDict, getUserList

In [ ]:
phDct = loadUserPhonenumberDict()
userLst = getUserList()

In [ ]:
u0 = userLst[0]

In [ ]:
userDct = loadUser(u0, dataFilter=("call", "sms"))

In [ ]:
callDct, smsDct = defaultdict(Counter), defaultdict(Counter)

In [ ]:
def linkUser(userDct, linkDctLst, phDct):
    """Links users in the project with each other.
    
    Args:
        userDct (dict): Dictionary with sms and call entries.
        linkDctLst (defaultdict(Counter)): Defaultdict with counter-factory
                                           to provide history.
        phDct (dict): Dictionary with phoneID -> userID for participants in the project.
        
    Returns:
        (float, float): Fraction of (calls, sms) to participants in the project.
    """
    def updateDict(commLst, linkDct, numberIndexStr):
        inProjectCEvent = 0
        for event in commLst:
            calledNumber = event[numberIndexStr]
            user = event["user"]
            if calledNumber in phDct:
                inProjectCEvent += 1
                calledUser = phDct[calledNumber]
                linkDct[user][calledUser] += 1
        return inProjectCEvent

    numberTypePairTuple = (("call", "number"), ("sms", "address"))
    inProjectFractions = list()
    for (commType, numberIndexStr), linkDct in zip(numberTypePairTuple, linkDctLst):
        count = updateDict(userDct[commType], linkDct, numberIndexStr)
        inProjectFractions.append(count)
    inProjectFractions[0] /= len(userDct[commType])
    inProjectFractions[1] /= len(userDct[commType])
    return inProjectFractions

In [ ]:
linkUser(userDct, (callDct, smsDct), phDct)

In [ ]:
callDct

In [ ]:
smsDct